In [1]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import time
import requests

In [2]:
# Set the executable path and initialize the chrome browser in splinter
executable_path = {'executable_path': 'chromedriver'}
browser = Browser('chrome', **executable_path)

In [3]:
# Visit the mars nasa news site
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)

True

In [4]:
html = browser.html
news_soup = BeautifulSoup(html, 'html.parser')
slide_elem = news_soup.select_one('ul.item_list li.slide')

In [5]:
# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find("div", class_='content_title').get_text()
news_title

"Nine Finalists Chosen in NASA's Mars 2020 Rover Naming Contest"

In [6]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_="article_teaser_body").get_text()
news_p

"Nine finalists have been chosen in the essay contest for K-12 students across U.S. to name NASA's next Mars rover. Now you can help by voting for your favorite. "

### Featured Images

In [7]:
# Visit URL
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [8]:
# Find and click the full image button
full_image_elem = browser.find_by_id('full_image')
full_image_elem.click()

In [9]:
# Find the more info button and click that
browser.is_element_present_by_text('more info', wait_time=1)
more_info_elem = browser.find_link_by_partial_text('more info')
more_info_elem.click()

C:\ProgramData\Anaconda3\lib\site-packages\splinter\driver\webdriver\__init__.py:528: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


In [10]:
# Parse the resulting html with soup
html = browser.html
img_soup = BeautifulSoup(html, 'html.parser')

In [11]:
# Find the relative image url
img_url_rel = img_soup.select_one('figure.lede a img').get("src")
img_url_rel

'/spaceimages/images/largesize/PIA17009_hires.jpg'

In [12]:
# Use the base URL to create an absolute URL
img_url = f'https://www.jpl.nasa.gov{img_url_rel}'
img_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17009_hires.jpg'

In [13]:
df = pd.read_html('http://space-facts.com/mars/')[0]
df.columns=['description', 'value']
df.set_index('description', inplace=True)
df

,value
description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


For Challenge
1.Obtain high-resolution images for each of Mar’s hemispheres.
    Note: You must click each hemisphere’s link to access the full-resolution image’s URL.
2.Save both the image URL string (for the full-resolution image) and the hemisphere title (with the name).
3.Use a Python dictionary to store the data using the keys `img_url` and `title.`
4.Append the dictionary with the image URL string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

In [14]:
hemisphere_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemisphere_url)

In [15]:
hemisphere_html = browser.html
soup = BeautifulSoup(hemisphere_html, 'html.parser')

In [16]:
items = soup.find_all('div', class_='item')
hemisphere_image_urls = []
hemispheres_main_url = 'https://astrogeology.usgs.gov'

for item in items:    
    title = item.find('h3').text    
    image_url = item.find('a', class_='itemLink product-item')['href']
    browser.visit(hemispheres_main_url + image_url)
    image_html = browser.html
    soup = BeautifulSoup( image_html, 'html.parser')
    image_url = hemispheres_main_url + soup.find('img', class_='wide-image')['src']
    hemisphere_image_urls.append({"Title" : title, "Img_URL" : image_url})

In [17]:
print(f"Hemisphere Image URLs")
print()
print("-------------------------------------------------")
print()
print(f"Cerberus Hemisphere Enhanced:\n{hemisphere_image_urls[0]}")
print()
print(f"Schiaparelli Hemisphere Enhanced:\n{hemisphere_image_urls[1]}")
print()
print(f"Syrtis Major Hemisphere Enhanced:\n{hemisphere_image_urls[2]}")
print()
print(f"Valles Marineris Hemisphere Enhanced:\n{hemisphere_image_urls[3]}")

Hemisphere Image URLs

-------------------------------------------------

Cerberus Hemisphere Enhanced:
{'Title': 'Cerberus Hemisphere Enhanced', 'Img_URL': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'}

Schiaparelli Hemisphere Enhanced:
{'Title': 'Schiaparelli Hemisphere Enhanced', 'Img_URL': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'}

Syrtis Major Hemisphere Enhanced:
{'Title': 'Syrtis Major Hemisphere Enhanced', 'Img_URL': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'}

Valles Marineris Hemisphere Enhanced:
{'Title': 'Valles Marineris Hemisphere Enhanced', 'Img_URL': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}


In [18]:
hemisphere_image_urls = []

#Cerberus Hemispheres

In [24]:
url = ('https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced')
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
cerberus_img = soup.find('div', class_="wide-image-wrapper")
pic = cerberus_img.find('li')
full_img = pic.find('a')['href']
cerberus_title = soup.find('h2', class_='title').text
cerberus_hem = {"Title": cerberus_title, "url": full_img}
print (cerberus_hem)

{'Title': 'Cerberus Hemisphere Enhanced', 'url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}


#Schiaparelli Hemisphere

In [26]:
url = ('https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced')
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
schiaparelli_img = soup.find('div', class_="wide-image-wrapper")
pic = schiaparelli_img.find('li')
full_img = pic.find('a')['href']
schiaparelli_title = soup.find('h2', class_='title').text
schiaparelli_hem = {"Title": schiaparelli_title, "url": full_img}
print (schiaparelli_hem)

{'Title': 'Schiaparelli Hemisphere Enhanced', 'url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}


#Syrtis Hemisphere

In [27]:
url = ('https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced')
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
syrtis_img = soup.find('div', class_="wide-image-wrapper")
pic = syrtis_img.find('li')
full_img = pic.find('a')['href']
syrtis_title = soup.find('h2', class_='title').text
syrtis_hem = {"Title": syrtis_title, "url": full_img}
print (syrtis_hem)

{'Title': 'Syrtis Major Hemisphere Enhanced', 'url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}


#Valles Marineris Hemisphere

In [29]:
url = ('https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced')
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
vallesMarineris_img = soup.find('div', class_="wide-image-wrapper")
pic = vallesMarineris_img.find('li')
full_img = pic.find('a')['href']
vallesMarineris_title = soup.find('h2', class_='title').text
vallesMarineris_hem = {"Title": vallesMarineris_title, "url": full_img}
print (vallesMarineris_hem)

{'Title': 'Valles Marineris Hemisphere Enhanced', 'url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}


In [ ]:
#browser.quit()